In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
data = pd.read_csv('/content/play_tennis.csv')

In [60]:
data.head()

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [33]:
res = { }
for col in data.drop(columns=['day','play']).columns:
   d = {}
   for i in data.play.unique():
       tab = dict(data[data.play == i][col].value_counts())
       n = sum(tab.values())
       for j in data[col].unique():
           if j in tab:
              tab[j]/=n
           else:
              tab[j]=0
       d[i] = tab
   res[col]=d
print(res)

{'outlook': {'No': {'Sunny': 0.6, 'Rain': 0.4, 'Overcast': 0}, 'Yes': {'Overcast': 0.4444444444444444, 'Rain': 0.3333333333333333, 'Sunny': 0.2222222222222222}}, 'temp': {'No': {'Hot': 0.4, 'Mild': 0.4, 'Cool': 0.2}, 'Yes': {'Mild': 0.4444444444444444, 'Cool': 0.3333333333333333, 'Hot': 0.2222222222222222}}, 'humidity': {'No': {'High': 0.8, 'Normal': 0.2}, 'Yes': {'Normal': 0.6666666666666666, 'High': 0.3333333333333333}}, 'wind': {'No': {'Strong': 0.6, 'Weak': 0.4}, 'Yes': {'Weak': 0.6666666666666666, 'Strong': 0.3333333333333333}}}


In [51]:
def outlook(i):
  if i == 'Sunny':
    return 0
  elif i == 'Overcast':
    return 1
  elif i == 'Rain' :
    return 2
  elif i == 'Hot':
    return 0
  elif i == 'Mild':
    return 1
  elif i == 'Cool':
    return 2
  elif i == 'High':
    return 0
  elif i == 'Normal':
    return 1
  elif i == 'Strong':
    return 0
  elif i == 'Weak':
    return 1
  elif i == 'No':
    return 0
  elif i == 'Yes':
    return 1
  else:
    return i
for i in data.columns:
    data[i]=data[i].apply(outlook)

In [61]:
data.drop(columns=['day'],inplace=True)

In [57]:
X = data.drop(columns=["play"]).values
y = data.play.values

In [62]:
res = { }
target = 'play'
for col in data.drop(columns=[target]).columns:
   d = {}
   for i in data[target].unique():
       tab = dict(data[data[target] == i][col].value_counts())
       n = sum(tab.values())
       for j in data[col].unique():
           if j in tab:
              tab[j]/=n
           else:
              tab[j]=0
       d[i] = tab
   res[col]=d
print(res)

{'outlook': {'No': {'Sunny': 0.6, 'Rain': 0.4, 'Overcast': 0}, 'Yes': {'Overcast': 0.4444444444444444, 'Rain': 0.3333333333333333, 'Sunny': 0.2222222222222222}}, 'temp': {'No': {'Hot': 0.4, 'Mild': 0.4, 'Cool': 0.2}, 'Yes': {'Mild': 0.4444444444444444, 'Cool': 0.3333333333333333, 'Hot': 0.2222222222222222}}, 'humidity': {'No': {'High': 0.8, 'Normal': 0.2}, 'Yes': {'Normal': 0.6666666666666666, 'High': 0.3333333333333333}}, 'wind': {'No': {'Strong': 0.6, 'Weak': 0.4}, 'Yes': {'Weak': 0.6666666666666666, 'Strong': 0.3333333333333333}}}


In [64]:
def pridict(inp):
  col = data.drop(columns=[target]).columns
  r = { }
  for i in np.unique(data[target]):
    prob = 1
    for j in range(len(inp)):
      prob*=res[col[j]][i][inp[j]]
    r[i]=prob
  return max(r,key=r.get)
pridict(['Sunny','Hot','High','Weak'])

'No'

In [74]:
class NaiveBayes:

  def __init__(self):
    self.dic = { }
    self.target = None

  def fit (self,X,target):
    self.target = target
    for col in data.drop(columns=[self.target]).columns:
       d = {}
       for i in data[self.target].unique():
           tab = dict(data[data[self.target] == i][col].value_counts())
           n = sum(tab.values())
           for j in data[col].unique():
               if j in tab:
                   tab[j]/=n
               else:
                   tab[j]=0
           d[i] = tab
       self.dic[col]=d
    for i in np.unique(data[self.target]):
      self.dic[i] = data[data[self.target]==i][target].count()/data[self.target].count()
    return self.dic

  def pridict(self,inp):
    col = data.drop(columns=[target]).columns
    r = { }
    for i in np.unique(data[target]):
        prob = self.dic[i]
        for j in range(len(inp)):
            prob*=res[col[j]][i][inp[j]]
        r[i]=prob
    return max(r,key=r.get)

In [75]:
Model = NaiveBayes()
Model.fit(data,'play')

{'outlook': {'No': {'Sunny': 0.6, 'Rain': 0.4, 'Overcast': 0},
  'Yes': {'Overcast': 0.4444444444444444,
   'Rain': 0.3333333333333333,
   'Sunny': 0.2222222222222222}},
 'temp': {'No': {'Hot': 0.4, 'Mild': 0.4, 'Cool': 0.2},
  'Yes': {'Mild': 0.4444444444444444,
   'Cool': 0.3333333333333333,
   'Hot': 0.2222222222222222}},
 'humidity': {'No': {'High': 0.8, 'Normal': 0.2},
  'Yes': {'Normal': 0.6666666666666666, 'High': 0.3333333333333333}},
 'wind': {'No': {'Strong': 0.6, 'Weak': 0.4},
  'Yes': {'Weak': 0.6666666666666666, 'Strong': 0.3333333333333333}},
 'No': 0.35714285714285715,
 'Yes': 0.6428571428571429}

In [83]:
Model.pridict(list(data.iloc[5,:-1]))

'Yes'

In [84]:
c = 0
inc = 0
for i in range(len(data)):
  if Model.pridict(list(data.iloc[i,:-1])) == data.iloc[i,-1]:
    c+=1
  else:
    inc+=1
print(c/(c+inc))

0.9285714285714286
